In [5]:
from qiskit import IBMQ
import numpy as np
from numpy import linalg
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute #, Aer
from qiskit import execute, IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from tools import set_parameters
from launcher import Launcher
import cmath
import math
from scipy.linalg import expm

parameters = {}
parameters['backend_name'] = 'qasm_simulator'
parameters['shots'] = 100
launcher = Launcher(**{item: parameters[item] for item in ['backend_name', 'shots']})
#ROTATION
def ConditionedRotation(angle, controllingQbit, subjectedQbit, circuit, quantReg):
  
    circuit.u3(angle, 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    circuit.cx(quantReg[controllingQbit], quantReg[subjectedQbit])
    circuit.u3( ( - angle ), 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])

t = 0.25    
n = 2
x = np.array([[2, 1], [1, 2]])

def get_abgd_from_unitary_matrix(U):
    if abs(U[0][0]) < 1e-4:
        gamma = np.pi
        delta = 0
        alpha = 0.5*cmath.phase(-U[1,0]*U[0,1])
        beta = 2*(cmath.phase(U[1,0]) - alpha)
    elif abs(U[1][0]) < 1e-4:
        gamma = 0
        delta = 0
        alpha = 0.5*cmath.phase(U[0,0]*U[1,1])
        beta = 2*(cmath.phase(U[1,1]) - alpha)
    else:
        alpha = 0.5*cmath.phase(U[0][0]*U[1][1])
        beta = cmath.phase(U[1][0]/U[0][0])
        gamma = 2*math.asin(abs(U[1][0]))
        delta = (cmath.phase(U[1][1]/U[0][0]) - beta)
    return alpha, beta, gamma, delta

y = expm(2*1j*np.pi*t*x)
a, b, c, d = get_abgd_from_unitary_matrix(y)

y_2 = np.array(y)@np.array(y)
a1, b1, c1, d1 = get_abgd_from_unitary_matrix(y_2)

    
solution = np.array([math.sqrt(0.5), math.sqrt(0.5)])
# solution = np.array([2, -1])/np.sqrt(5)
solution_T = np.conj(solution.T)


# Define the Quantum and Classical Registers
qr = QuantumRegister(5)
cr = ClassicalRegister(5)
# Build the circuits
circuit = QuantumCircuit(qr, cr)
#b
circuit.h(qr[4])

#U
circuit.h(qr[3])
circuit.u1((d - b)/2, qr[4]) #C
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u1(-(d+b)/2, qr[4])
circuit.u3(-c/2, 0, 0, qr[4])
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(c/2, 0, 0, qr[4])
circuit.u1(b, qr[4])
circuit.u1(a, qr[3]) #a

# U^2
circuit.h(qr[2])
circuit.u1((d1 - b1)/2, qr[4]) #C
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u1(-(d1+b1)/2, qr[4])
circuit.u3(-c1/2, 0, 0, qr[4])
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(c1/2, 0, 0, qr[4])
circuit.u1(b1, qr[4])
circuit.u1(a1, qr[2]) #a

# FT.inverse()
circuit.h(qr[2])
circuit.u1(-(np.pi/4), qr[2])
circuit.u1(-np.pi/2, qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.h(qr[3])


# from qiskit import execute

# circuit.measure(qr[2], cr[2])
# circuit.measure(qr[3], cr[3])
# print(execute(circuits=[circuit], backend=BasicAer.get_backend('qasm_simulator'), shots=8192).result().get_counts())
# import sys; sys.exit(0)


#R
ConditionedRotation(np.pi/4, 2, 1, circuit, qr)
ConditionedRotation(np.pi/8, 3, 1, circuit, qr)

#FT
circuit.h(qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(np.pi/2, qr[3])
circuit.u1((np.pi/4), qr[2])
circuit.h(qr[2])

circuit.u1(-a1, qr[2])
circuit.u1(-b1, qr[4])
circuit.u3(-c1/2, 0, 0, qr[4])
circuit.cx(qr[2], qr[4])
circuit.u3(c1/2, 0, 0, qr[4])
circuit.u1((d1+b1)/2, qr[4])
circuit.cx(qr[2], qr[4])
circuit.u1(-(d1 - b1)/2, qr[4]) #C
circuit.h(qr[2])


circuit.u1(-a, qr[3])
circuit.u1(-b, qr[4])
circuit.u3(-c/2, 0, 0, qr[4])
circuit.cx(qr[3], qr[4])
circuit.u3(c/2, 0, 0, qr[4])
circuit.u1((d+b)/2, qr[4])
circuit.cx(qr[3], qr[4])
circuit.u1(-(d - b)/2, qr[4]) #C
circuit.h(qr[3])


# meas_qubits - кубиты, использующиеся для томографии. Порядок важен (для [0, 1] и [1, 0] ответ будет разным)
exp_solution_4 = launcher.run([circuit], meas_qubits=[1, 4])[0]
# print(exp_solution_4)

exp_solution_4 = np.kron(np.eye(2), np.diag([0, 1]))@exp_solution_4@np.kron(np.eye(2), np.diag([0, 1]))

exp_solution_4 = exp_solution_4/np.trace(exp_solution_4)

exp_solution_2 = np.array([
    [exp_solution_4[0,0] + exp_solution_4[1,1], exp_solution_4[0,2] + exp_solution_4[1,3]],
    [exp_solution_4[2,0] + exp_solution_4[3,1],exp_solution_4[2,2] + exp_solution_4[3,3]]
])

# print(exp_solution_2)
prob = solution @ exp_solution_2 @ solution_T
print(prob)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

(0.9760616040553522+0j)


In [3]:
circuit.draw()